In [1]:
import deepchem as dc
from rdkit import Chem
from rdkit.Chem.MolStandardize.rdMolStandardize import StandardizeSmiles
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

import numpy as np
import pandas as pd
import os

In [50]:
def neutralize_atoms(mol):
    pattern = Chem.MolFromSmarts("[+1!h0!$([*]~[-1,-2,-3,-4]),-1!$([*]~[+1,+2,+3,+4])]")
    at_matches = mol.GetSubstructMatches(pattern)
    at_matches_list = [y[0] for y in at_matches]
    if len(at_matches_list) > 0:
        for at_idx in at_matches_list:
            atom = mol.GetAtomWithIdx(at_idx)
            chg = atom.GetFormalCharge()
            hcount = atom.GetTotalNumHs()
            atom.SetFormalCharge(0)
            atom.SetNumExplicitHs(hcount - chg)
            atom.UpdatePropertyCache()
    return mol

def try_standardize(smi):
    try:
        return StandardizeSmiles(Chem.MolToSmiles(neutralize_atoms(Chem.MolFromSmiles(smi))))
    except:
        pass
    return 'fail'

def clean_up_smiles(df):
    df['smiles'] = df['smiles'].map(lambda s: try_standardize(s))
    df = df[df.smiles != 'fail']
    return df

def df_to_ds(df):
    X = np.zeros(shape=(len(df),1))
    return dc.data.DiskDataset.from_numpy(X=X, y=np.vstack(df.label.to_numpy()), ids=df.smiles)

In [51]:
path_to_pickle = 'datasets/small_dataset2_CHEMBL2114881.pkl'
out_dir_name = ''
splitter = 'random'
split_type = ''

df = pd.read_pickle(path_to_pickle)
df = clean_up_smiles(df)
ds = df_to_ds(df)

In [53]:
quit()

In [4]:
if splitter == 'random':
    splitter = dc.splits.RandomSplitter()
    split_type = 'random'
elif splitter == 'fingerprint':
    splitter = dc.splits.FingerprintSplitter()
    split_type = 'finger'
elif splitter == 'scaffold':
    splitter = dc.splits.ScaffoldSplitter()
    split_type = 'scafld'

In [5]:
train_test_splits = random.k_fold_split(dataset=ds, k=5)

In [7]:
out_dir_name = f'CHEMBL2114881_{split_type}_split'
os.mkdir(path=out_dir_name)

In [30]:
for i,tt in enumerate(train_test_splits):
    train_df = tt[0].to_dataframe()
    train_df = train_df.rename({'ids': 'smiles', 'y': 'label'}, axis='columns').drop(['X', 'w'], axis='columns')
    train_df.to_pickle(f'{out_dir_name}/train{i}')
    
    test_df = tt[1].to_dataframe()
    test_df = test_df.rename({'ids': 'smiles', 'y': 'label'}, axis='columns').drop(['X', 'w'], axis='columns')
    test_df.to_pickle(f'{out_dir_name}/test{i}')

In [27]:
test = []
for i in range(5):
    test.append((pd.read_pickle(f'{out_dir_name}/train{i}'), pd.read_pickle(f'{out_dir_name}/test{i}')))